In [ ]:
#!eval-notebook "../Report/ReportStorage"

In [ ]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

# Test Exchange Rate Query

In [ ]:
Workspace.Initialize(x => x.FromSource(DataSource)
         .DisableInitialization<ExchangeRate>());

In [ ]:
public async Task CheckGetFx(string currentCurrency, string targetCurrency, int year, int month, IEnumerable<ExchangeRate> testData, double fxBOPBenchmark, double fxAVGBenchmark, double fxEOPBenchmark)
{
    await Workspace.UpdateAsync(testData);
    
    //Create report storage
    var period = (year, month);
    var reportStorage = new ReportStorage(Workspace, Report);
    await reportStorage.InitializeReportIndependentCacheAsync();
    await reportStorage.InitializeAsync(period, "G", null, CurrencyType.Contractual);
    
    var fxBOP = reportStorage.GetFx(period, currentCurrency, targetCurrency, FxPeriod.BeginningOfPeriod);
    var fxAVG = reportStorage.GetFx(period, currentCurrency, targetCurrency, FxPeriod.Average);
    var fxEOP = reportStorage.GetFx(period, currentCurrency, targetCurrency, FxPeriod.EndOfPeriod);
    
    //Check FX rates
    fxBOP.Should().Be(fxBOPBenchmark);
    fxAVG.Should().Be(fxAVGBenchmark);
    fxEOP.Should().Be(fxEOPBenchmark);
    
    await Workspace.DeleteAsync(Workspace.Query<ExchangeRate>().ToArray());
}

In [ ]:
var testData = new ExchangeRate[] {new ExchangeRate{ Currency = "EUR", Year = 2020, Month = 12, FxType = FxType.Spot, FxToGroupCurrency = 10 }, 
                                 new ExchangeRate{ Currency = "EUR", Year = 2021, Month = 3, FxType = FxType.Spot, FxToGroupCurrency = 15 },
                                 new ExchangeRate{ Currency = "EUR", Year = 2021, Month = 6, FxType = FxType.Average, FxToGroupCurrency = 20 },
                                 new ExchangeRate{ Currency = "EUR", Year = 2021, Month = 6, FxType = FxType.Spot, FxToGroupCurrency = 30 },
                                 new ExchangeRate{ Currency = "USD", Year = 2020, Month = 12, FxType = FxType.Spot, FxToGroupCurrency = 5 },
                                 new ExchangeRate{ Currency = "USD", Year = 2021, Month = 6, FxType = FxType.Average, FxToGroupCurrency = 2 },
                                 new ExchangeRate{ Currency = "USD", Year = 2021, Month = 6, FxType = FxType.Spot, FxToGroupCurrency = 0.5 }};
await CheckGetFx("EUR", "USD", 2021, 6, testData, 2, 10, 60);

In [ ]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());